In [1]:
import datajoint as dj
import datetime
import os, glob
import re
import pandas as pd
import numpy as np
from RDBMS_tables import *


[2023-03-13 13:29:42,765][INFO]: Connecting root@localhost:3306
[2023-03-13 13:29:42,779][INFO]: Connected root@localhost:3306


### Clearing all columns before starting? - our custom .py file does this

Populating with mice

In [2]:
data = [
    {'mouse_id': 'BLA-Insc-1'},
    {'mouse_id': 'BLA-Insc-2'},
    {'mouse_id': 'BLA-Insc-3'},
    {'mouse_id': 'BLA-Insc-5'},
    {'mouse_id': 'BLA-Insc-6'},
    {'mouse_id': 'BLA-Insc-7'},
    {'mouse_id': 'BLA-Insc-8'},
    {'mouse_id': 'BLA-Insc-9'},
    {'mouse_id': 'BLA-Insc-11'},
    {'mouse_id': 'BLA-Insc-13'},
    {'mouse_id': 'BLA-Insc-14'},
    {'mouse_id': 'BLA-Insc-15'},
    {'mouse_id': 'BLA-Insc-16'},
    {'mouse_id': 'BLA-Insc-18'},
    {'mouse_id': 'BLA-Insc-19'}
]

mouse.insert(data, skip_duplicates=True)

## Making simple queries

In [3]:
query = mouse.fetch()
query

array([('BLA-Insc-1',), ('BLA-Insc-11',), ('BLA-Insc-13',),
       ('BLA-Insc-14',), ('BLA-Insc-15',), ('BLA-Insc-16',),
       ('BLA-Insc-18',), ('BLA-Insc-19',), ('BLA-Insc-2',),
       ('BLA-Insc-3',), ('BLA-Insc-5',), ('BLA-Insc-6',), ('BLA-Insc-7',),
       ('BLA-Insc-8',), ('BLA-Insc-9',)], dtype=[('mouse_id', 'O')])

In [4]:
(mouse & 'mouse_id = "BLA-Insc-1"').fetch()

array([('BLA-Insc-1',)], dtype=[('mouse_id', 'O')])

## Visualizing dependency

In [5]:
session = Session()
session

mouse_id unique mouse id,session_id session id,experiment_name experiment name
BLA-Insc-1,01142021,RM D10
BLA-Insc-1,01182021,Pre-RDT RM
BLA-Insc-1,01202021,RDT D1
BLA-Insc-1,01212021,Post-RDT D1
BLA-Insc-1,01262021,RDT D2
BLA-Insc-1,01272021,Post-RDT D2
BLA-Insc-1,02022021,Post-RDT D3
BLA-Insc-1,02082021,PR D1
BLA-Insc-1,02092021,PR D2
BLA-Insc-1,02112021,Late Shock D1


### Renaming a column - using "proj" operator

In [6]:
#session = session.proj(experiment_name = 'experiment_id')
#session

In [7]:
#session.drop_attribute('experiment_id')

In [8]:
mouse

mouse_id unique mouse id
BLA-Insc-1
BLA-Insc-11
BLA-Insc-13
BLA-Insc-14
BLA-Insc-15
BLA-Insc-16
BLA-Insc-18
BLA-Insc-19
BLA-Insc-2
BLA-Insc-3


## Let's try to add all the sessions for all the mice we have thus far
https://tutorials.datajoint.org/beginner/building-first-pipeline/python/child-table.html

In [19]:
def create_session_data_to_upload(ROOT, SUBDIRS, cols_to_fill, pattern, pattern_2, session_types) -> list:

    data = []
    for subdir in SUBDIRS:
        subdir_path = os.path.join(ROOT, subdir)
        # now will need to loop through the folders in the subdir to extract the mouse id
        for mouse_folder_name in os.listdir(subdir_path):
            print("mouse: ", mouse_folder_name)
            mouse_folder_path = os.path.join(subdir_path, mouse_folder_name)
            # now will need to loop through the folders in the mouse folder to extract the session date and experiment name
            print(mouse_folder_path)
            for session_folder_name in os.listdir(mouse_folder_path):
                print("session: ", session_folder_name)
                # example: /media/rory/Padlock_DT/BLA_Analysis/PTP_Inscopix_#1/BLA-Insc-1/Post-RDT D2/BLA-Insc-1 01272021_3_ABET_GPIO_processed.csv

                # first figure out if session is a session type we want to include
                if session_folder_name in session_types:
                    subdata = {}
                    
                    subdata[cols_to_fill[0]] = mouse_folder_name

                    # statically search for the file that will give you the date in the session folder
                    session_path = os.path.join(mouse_folder_path, session_folder_name)
                    found_date = False
                    for file in os.listdir(session_path):
                        if file.endswith(pattern):
                            found_date = True
                            print("found file: ", file)
                            date_str = re.search(r"\d{8}", file).group()
                            print("found date: ", date_str)
                            subdata[cols_to_fill[1]] = date_str
                            break
                    
                    # There are mice that what a subdir before you get there
                    if found_date == False:
                        for subdir_2 in os.listdir(session_path):
                            if subdir_2.startswith(pattern_2):
                                print("found subdir: ", subdir_2)
                                subdir_2_path = os.path.join(session_path, subdir_2)
                                for file in os.listdir(subdir_2_path):
                                    if file.endswith(pattern):
                                        found_date = True
                                        print("found file: ", file)
                                        date_str = re.search(r"\d{8}", file).group()
                                        print("found date: ", date_str)
                                        subdata[cols_to_fill[1]] = date_str
                                        break

                    # if actually not there then just put nan
                    if found_date == False:
                        print("date not found")
                        subdata[cols_to_fill[1]] = "nan"
                    
                    subdata[cols_to_fill[2]] = session_folder_name
                        
                    data.append(subdata)

                # what to do if the pattern is not found on a session_folder_name? What to do if date_str is not found?
    
    return data




In [20]:
ROOT = "/media/rory/Padlock_DT/BLA_Analysis"

SUBDIRS = ["PTP_Inscopix_#1", "PTP_Inscopix_#3", "PTP_Inscopix_#4", "PTP_Inscopix_#5"]

cols_to_fill = ['mouse_id', 'session_id', 'experiment_name']

pattern = r"ABET_processed.csv"
pattern_2 = r"Session"

session_types = [
            "PR D1",
            "PR D2",
            "Pre-RDT RM",
            "RDT D1",
            "RDT D2",
            "RDT D3",
            "Post-RDT D1",
            "Post-RDT D2",
            "Post-RDT D3",
            "RM D1",
            "RM D2",
            "RM D3",
            "RM D8",
            "RM D9",
            "RM D10",
            "Late Shock D1",
            "Late Shock D2",
        ]

data = create_session_data_to_upload(ROOT, SUBDIRS, cols_to_fill,  pattern, pattern_2, session_types)

print(data)      
session.insert(data, skip_duplicates=True)
session

mouse:  BLA-Insc-1
/media/rory/Padlock_DT/BLA_Analysis/PTP_Inscopix_#1/BLA-Insc-1
session:  .~lock.longitudinal_registration.csv#
session:  cellreg_Pre-RDT RM_RDT D1.csv
session:  Late Shock D1
found file:  BLA-Insc-1 02112021_3_ABET_processed.csv
found date:  02112021
session:  Late Shock D2
found file:  BLA-Insc-1 02152021_3_ABET_processed.csv
found date:  02152021
session:  longreg_results.csv
session:  longreg_results_preprocessed.csv
session:  Post-RDT D1
found file:  BLA-Insc-1 01212021_ABET_processed.csv
found date:  01212021
session:  Post-RDT D2
found file:  BLA-Insc-1 01272021_3_ABET_processed.csv
found date:  01272021
session:  Post-RDT D3
found file:  BLA-Insc-1 02022021_3_ABET_processed.csv
found date:  02022021
session:  PR D1
found file:  BLA-Insc-1 02082021_3_ABET_processed.csv
found date:  02082021
session:  PR D2
found file:  BLA-Insc-1 02092021_3_ABET_processed.csv
found date:  02092021
session:  Pre-RDT RM
found file:  BLA-Insc-1 01182021_ABET_processed.csv
found da

[2023-03-14 10:44:24,830][WARNING]: MySQL server has gone away. Reconnecting to the server.


found file:  BLA-Insc-19 01242022_ABET_processed.csv
found date:  01242022
session:  RDT D1
found subdir:  Session-20220125-115633_BLA-Insc-19_RDT_D
found file:  BLA-Insc-19 01252022_ABET_processed.csv
found date:  01252022
session:  RDT D2
found subdir:  Session-20220131-115917_BLA-Insc-19_RDT_D2
found file:  BLA-Insc-19 01312022_ABET_processed.csv
found date:  01312022
session:  RDT D3
found subdir:  Session-20220204-114235_BLA-INSC-19_RDT_D3
found file:  BLA-Insc-19 02042022_ABET_processed.csv
found date:  02042022
session:  RM D1
found subdir:  Session-20220106-115100_BLA-INSC-19_RM_D1
found file:  BLA-Insc-19 01062022_ABET_processed.csv
found date:  01062022
session:  Shock Test
[{'mouse_id': 'BLA-Insc-1', 'session_id': '02112021', 'experiment_name': 'Late Shock D1'}, {'mouse_id': 'BLA-Insc-1', 'session_id': '02152021', 'experiment_name': 'Late Shock D2'}, {'mouse_id': 'BLA-Insc-1', 'session_id': '01212021', 'experiment_name': 'Post-RDT D1'}, {'mouse_id': 'BLA-Insc-1', 'session_id

mouse_id unique mouse id,session_id session id,experiment_name experiment name
BLA-Insc-1,01142021,RM D10
BLA-Insc-1,01182021,Pre-RDT RM
BLA-Insc-1,01202021,RDT D1
BLA-Insc-1,01212021,Post-RDT D1
BLA-Insc-1,01262021,RDT D2
BLA-Insc-1,01272021,Post-RDT D2
BLA-Insc-1,02022021,Post-RDT D3
BLA-Insc-1,02082021,PR D1
BLA-Insc-1,02092021,PR D2
BLA-Insc-1,02112021,Late Shock D1


In [23]:
session & "mouse_id = 'BLA-Insc-1'"

mouse_id unique mouse id,session_id session id,experiment_name experiment name
BLA-Insc-1,01142021,RM D10
BLA-Insc-1,01182021,Pre-RDT RM
BLA-Insc-1,01202021,RDT D1
BLA-Insc-1,01212021,Post-RDT D1
BLA-Insc-1,01262021,RDT D2
BLA-Insc-1,01272021,Post-RDT D2
BLA-Insc-1,02022021,Post-RDT D3
BLA-Insc-1,02082021,PR D1
BLA-Insc-1,02092021,PR D2
BLA-Insc-1,02112021,Late Shock D1


In [21]:
session_df = session.fetch()
session_df

array([('BLA-Insc-1', '01142021', 'RM D10'),
       ('BLA-Insc-1', '01182021', 'Pre-RDT RM'),
       ('BLA-Insc-1', '01202021', 'RDT D1'),
       ('BLA-Insc-1', '01212021', 'Post-RDT D1'),
       ('BLA-Insc-1', '01262021', 'RDT D2'),
       ('BLA-Insc-1', '01272021', 'Post-RDT D2'),
       ('BLA-Insc-1', '02022021', 'Post-RDT D3'),
       ('BLA-Insc-1', '02082021', 'PR D1'),
       ('BLA-Insc-1', '02092021', 'PR D2'),
       ('BLA-Insc-1', '02112021', 'Late Shock D1'),
       ('BLA-Insc-1', '02152021', 'Late Shock D2'),
       ('BLA-Insc-1', 'nan', 'RDT D3'),
       ('BLA-Insc-11', '10202021', 'Pre-RDT RM'),
       ('BLA-Insc-11', '10212021', 'RDT D1'),
       ('BLA-Insc-11', '10292021', 'RDT D2'),
       ('BLA-Insc-11', '11032021', 'RDT D3'),
       ('BLA-Insc-11', '11082021', 'PR D1'),
       ('BLA-Insc-11', '11092021', 'PR D2'),
       ('BLA-Insc-11', 'nan', 'PR D1'),
       ('BLA-Insc-13', '10122021', 'RM D1'),
       ('BLA-Insc-13', '10282021', 'Pre-RDT RM'),
       ('BLA-Insc-13'

### Tried to have the DB drawn on a diagram
- not needed ig cuz have it on mysql workbench

In [ ]:
dj.ERD(schema).draw()

### Exporting Schema to a .db file 
Not needed, got it on MySQL Workbench and MySQL CLI

### Now add certain cells